In [ ]:
!pip install opencv-python numpy ultralytics

In [ ]:
# CODE CELL 1: Create the fixed tracker config file
yaml_content = """
tracker_type: 'bytetrack'
track_high_thresh: 0.3
track_low_thresh: 0.1
new_track_thresh: 0.7
track_buffer: 100
match_thresh: 0.75
fuse_score: True
"""

# Write the file to the current working directory (/kaggle/working)
with open('tracker_config.yaml', 'w') as f:
    f.write(yaml_content)

print("tracker_config.yaml has been created and saved to the current working directory.")

tracker_config.yaml has been created and saved to the current working directory.


In [ ]:
import cv2

cap = cv2.VideoCapture('shortened_lab.mp4')
if not cap.isOpened():
    print("Error: Could not open video file. Check the path to 'input_video.mp4'.")

In [ ]:
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

from ultralytics import YOLO # Library for YOLOv8 Object Detection

# --- 1. HOMOGRAPHY SETUP (CALIBRATION) ---

# Define the coordinates that link your video feed to your 12x6 map.
# *YOU MUST REPLACE THESE WITH YOUR REAL-WORLD COORDINATES*
VIDEO_PTS_SRC = np.array([
    [10, 320],  # Corresponds to Top-Left in a flat map view
    [840, 320], # Corresponds to Top-Right
    [850, 479], # Corresponds to Bottom-Right
    [0, 479]   # Bottom-Left
], dtype=np.float32)

# Map dimensions (e.g., 600x300 pixels to represent your 12x6 area)
MAP_WIDTH, MAP_HEIGHT = 850, 200
MAP_PTS_DST = np.array([
    [50, 20],   # P1: Top-Left in map (Y=20)
    [800, 20],  # P2: Top-Right (Y=20)
    [800, 180], # P3: Bottom-Right (Y=180)
    [50, 180] # Corresponds to Bottom-Left
], dtype=np.float32)

# Calculate Homography Matrix (H) once
H, _ = cv2.findHomography(VIDEO_PTS_SRC, MAP_PTS_DST)

# Map/Visualization Setup
TRACE_POINTS = {} # Dictionary to store trace points for multiple people (ID: [points])
MAP_IMAGE = np.zeros((MAP_HEIGHT, MAP_WIDTH, 3), dtype=np.uint8)

# --- 2. YOLOv8 MODEL & VIDEO SETUP ---

# Load the pre-trained YOLOv8s model (as used in the paper [cite: 16])
model = YOLO('yolo12n.pt')

# Replace 'input_video.mp4' with the path to your video file
cap = cv2.VideoCapture('shortened_lab.mp4')
if not cap.isOpened():
    print("Error: Could not open video file. Check the path to 'input_video.mp4'.")
    exit()

# Set up VideoWriter to save the result
FPS = cap.get(cv2.CAP_PROP_FPS) or 20
VIDEO_FILENAME = 'yolo_mapped_path.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter(VIDEO_FILENAME, fourcc, FPS, (MAP_WIDTH, MAP_HEIGHT))

print("Starting YOLO tracking and mapping simulation (Saving to video)...")

# --- 3. MAIN PROCESSING LOOP ---

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- A. OBJECT DETECTION & TRACKING (YOLO) ---
    # The 'persist=True' and 'tracker="bytetrack.yaml"' enable multi-object tracking
    # YOLO automatically assigns a unique 'track_id' to each detected person.
    results = model.track(frame, persist=True, tracker="tracker_config.yaml", classes=[0], verbose=False)

    # Reset map image for the current frame
    MAP_IMAGE = np.zeros((MAP_HEIGHT, MAP_WIDTH, 3), dtype=np.uint8)

    # 1. Highlight the Mapped Area Boundary
    cv2.polylines(MAP_IMAGE, [MAP_PTS_DST.astype(np.int32).reshape((-1, 1, 2))], isClosed=True, color=(255, 255, 0), thickness=2)

    # Check if tracking data is available
    if results[0].boxes.id is not None:
        boxes = results[0].boxes.xywh.cpu().numpy()
        track_ids = results[0].boxes.id.int().cpu().numpy()

        for box, track_id in zip(boxes, track_ids):
            # Bounding Box (x_center, y_center, w, h)
            x_c, y_c, w, h = box
            x1 = int(x_c - w/2)
            y1 = int(y_c - h/2)
            x2 = int(x_c + w/2)
            y2 = int(y_c + h/2)

            # TRUE feet position = midpoint of the bottom edge of the box
            feet_x_cam = int((x1 + x2) / 2)
            feet_y_cam = int(y2)

            # --- B. HOMOGRAPHY TRANSFORMATION ---
            camera_point = np.array([[feet_x_cam, feet_y_cam]], dtype=np.float32)
            mapped_point = cv2.perspectiveTransform(np.array([camera_point]), H)

            map_x, map_y = mapped_point[0][0].astype(int)

            # 3. STORE AND DRAW TRACE
            track_id_str = str(track_id)
            if track_id_str not in TRACE_POINTS:
                TRACE_POINTS[track_id_str] = []

            # Add the current position to the trace history for this person
            TRACE_POINTS[track_id_str].append((map_x, map_y))

            # Draw the full path for this person
            path = TRACE_POINTS[track_id_str]
            if len(path) > 1:
                # Use a different color for each person's path (based on ID)
                trail_color_code = int(track_id) * 50 % 255
                trail_color = (trail_color_code, 255 - trail_color_code, 100)

                points_array = np.array(path, np.int32)
                cv2.polylines(MAP_IMAGE, [points_array.reshape((-1, 1, 2))], isClosed=False, color=trail_color, thickness=2)

            # 4. DRAW CURRENT POSITION (Dot) on the map
            cv2.circle(MAP_IMAGE, (map_x, map_y), radius=6, color=(0, 0, 255), thickness=-1)
            cv2.putText(MAP_IMAGE, str(track_id), (map_x + 8, map_y - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

            # Optional: Draw on the source video frame for verification
            cv2.circle(frame, (feet_x_cam, feet_y_cam), 5, (0, 255, 255), -1)
            cv2.putText(frame, f"ID:{track_id}", (feet_x_cam, feet_y_cam - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)

    # --- C. WRITE AND DISPLAY ---
    video_writer.write(MAP_IMAGE)

    cv2_imshow(MAP_IMAGE)
    cv2_imshow(frame)